<a href="https://colab.research.google.com/github/Gauravsharma-20/Minor-Project/blob/master/Pred_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Important Libraries

In [ ]:
import tensorflow as tf
import h5py
import shutil
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout
from keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
import imutils
import time
from torchvision import datasets, models, transforms
import cv2
from keras.models import Input,Model
from tqdm.notebook import tqdm
from keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm.notebook import tqdm
from tensorflow.python.framework.ops import Tensor
from keras.layers import Conv2D,Dropout
from keras.layers import Activation,Average
from keras.layers import MaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam
import collections
from keras.losses import categorical_crossentropy

In [ ]:
#mount drive here
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.getcwd()+'/gdrive'

'/content/gdrive'

In [ ]:
#val dataset is downloaded into drive

In [ ]:
#splitting val to train test

In [ ]:
!pip install split-folders

In [ ]:
!mkdir out_val

In [ ]:
import splitfolders
splitfolders.ratio(os.getcwd()+'/gdrive/My Drive/val', output='out_val', seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 696 files [03:14,  3.59 files/s]


LOAD PREPROCESSED DATASET 

In [ ]:
X = []
y = []

inputDirectory = '/content/out_val/train/0'

for filename in tqdm(os.listdir(inputDirectory)):
    img=inputDirectory+'/'+filename
    img= cv2.imread(img)
    img= np.array(img)
    X.append(img)
    y.append(0)
inputDirectory = '/content/out_val/train/1'
for filename in tqdm(os.listdir(inputDirectory)):
    img=inputDirectory+'/'+filename
    img= cv2.imread(img)
    img= np.array(img)
    X.append(img)
    y.append(1)
inputDirectory = '/content/out_val/train/2'
for filename in tqdm(os.listdir(inputDirectory)):
    img=inputDirectory+'/'+filename
    img= cv2.imread(img)
    img= np.array(img)
    X.append(img)
    y.append(2)
inputDirectory = '/content/out_val/train/3'
for filename in tqdm(os.listdir(inputDirectory)):
    img=inputDirectory+'/'+filename
    img= cv2.imread(img)
    img= np.array(img)
    X.append(img)
    y.append(3)
inputDirectory = '/content/out_val/train/4'
for filename in tqdm(os.listdir(inputDirectory)):
    img=inputDirectory+'/'+filename
    img= cv2.imread(img)
    img= np.array(img)
    X.append(img)
    y.append(4)
X = np.array(X) 


In [ ]:
print(X.shape)
print(len(y))

(554, 124, 224, 3)
554


In [ ]:
#number of instances of each class
collections.Counter(y)  


Counter({0: 226, 1: 111, 2: 128, 3: 68, 4: 21})

In [ ]:
y=to_categorical(y)

In [ ]:
#-------splitting dataset---------- 

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=78)

In [ ]:
#X[0]
#y

INput Shape

In [ ]:
"""import cv2

im = cv2.imread(X[0])"""

'import cv2\n\nim = cv2.imread(X[0])'

In [ ]:
X = np.array(X)
input_for_model=Input(shape=(None,124,224,3))

In [ ]:
input_shape=(None,124,224,3)


ConvPool_CNN_C Model

In [ ]:
#ConvPool_CNN_C 
def ConvPool_CNN_C(X,y,learningRate=0.001,lossFunction='categorical_crossentropy'):
  model = Sequential()
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(MaxPooling2D(pool_size=(3,3),strides=2))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(MaxPooling2D(pool_size=(3,3),strides=2))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(1,1),activation='relu'))
  model.add(Conv2D(5,(1,1)))
  model.add(GlobalAveragePooling2D())
  model.add(Flatten())
  model.add(Dense(5, activation='softmax'))
  model.build(input_shape)
  model.compile(loss=categorical_crossentropy,optimizer=Adam(0.001),metrics=['accuracy'])
  return model

In [ ]:
model = ConvPool_CNN_C(X, y)

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 124, 224, 96)      2688      
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 124, 224, 96)      83040     
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 124, 224, 96)      83040     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 61, 111, 96)       0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 61, 111, 192)      166080    
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 61, 111, 192)      331968    
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 61, 111, 192)     

In [ ]:
filepath = 'ConvPool_model.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_frequency=1)
print(X_train.shape, Y_train.shape)

(443, 124, 224, 3) (443, 5)


In [ ]:
history = model.fit(X_train, Y_train,batch_size = 2,epochs = 25,verbose = 1,validation_split = 0.2,validation_data = (X_test, Y_test),callbacks = [checkpoint],shuffle=True)

Epoch 1/25
177/177 [==============================] - ETA: 0s - loss: 1.6324 - accuracy: 0.3814
Epoch 00001: val_loss improved from inf to 1.36219, saving model to ConvPool_model.hdf5
177/177 [==============================] - 5s 28ms/step - loss: 1.6324 - accuracy: 0.3814 - val_loss: 1.3622 - val_accuracy: 0.4944
Epoch 2/25
177/177 [==============================] - ETA: 0s - loss: 1.5178 - accuracy: 0.3701
Epoch 00002: val_loss did not improve from 1.36219
177/177 [==============================] - 5s 26ms/step - loss: 1.5178 - accuracy: 0.3701 - val_loss: 1.4094 - val_accuracy: 0.4944
Epoch 3/25
177/177 [==============================] - ETA: 0s - loss: 1.4829 - accuracy: 0.3842
Epoch 00003: val_loss improved from 1.36219 to 1.35144, saving model to ConvPool_model.hdf5
177/177 [==============================] - 5s 26ms/step - loss: 1.4829 - accuracy: 0.3842 - val_loss: 1.3514 - val_accuracy: 0.4944
Epoch 4/25
177/177 [==============================] - ETA: 0s - loss: 1.4544 - accura

In [ ]:
model.evaluate(X_test, Y_test, verbose=0)

[1.4009755849838257, 0.4054054021835327]

ALL_CNN_C Model

In [ ]:
def all_cnn_c(X,y,learningRate=0.001,lossFunction='categorical_crossentropy'):
  model = Sequential()
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(Conv2D(96,kernel_size=(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(192,(1,1),activation='relu'))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(5, activation='softmax'))
  model.build(input_shape)
  model.compile(loss=categorical_crossentropy,optimizer=Adam(0.001),metrics=['accuracy'])
  return model

In [ ]:
filepath = 'All_CNN_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_frequency=1)
print(X_train.shape, Y_train.shape)

(443, 124, 224, 3) (443, 5)


In [ ]:
all_cnn_c_model=all_cnn_c(X,y)

In [ ]:

history = all_cnn_c_model.fit(X_train, Y_train,batch_size = 2,epochs = 25,verbose = 1,validation_split = 0.2,validation_data = (X_test, Y_test),callbacks = [checkpoint],shuffle=True)

Epoch 1/25
177/177 [==============================] - ETA: 0s - loss: 1.8202 - accuracy: 0.3559
Epoch 00001: val_loss improved from inf to 1.49897, saving model to All_CNN_model.hdf5
177/177 [==============================] - 14s 79ms/step - loss: 1.8202 - accuracy: 0.3559 - val_loss: 1.4990 - val_accuracy: 0.4944
Epoch 2/25
177/177 [==============================] - ETA: 0s - loss: 1.4730 - accuracy: 0.3842
Epoch 00002: val_loss improved from 1.49897 to 1.41069, saving model to All_CNN_model.hdf5
177/177 [==============================] - 14s 76ms/step - loss: 1.4730 - accuracy: 0.3842 - val_loss: 1.4107 - val_accuracy: 0.4944
Epoch 3/25
177/177 [==============================] - ETA: 0s - loss: 1.4495 - accuracy: 0.3814
Epoch 00003: val_loss improved from 1.41069 to 1.38096, saving model to All_CNN_model.hdf5
177/177 [==============================] - 14s 77ms/step - loss: 1.4495 - accuracy: 0.3814 - val_loss: 1.3810 - val_accuracy: 0.4944
Epoch 4/25
177/177 [========================

NIN_CNN Model

In [ ]:
#nin_cnn_c model
def nin_cnn_c(X,y,learningRate=0.001,lossFunction='categorical_crossentropy'):
  model = Sequential()
  model.add(Conv2D(32,kernel_size=(5,5),activation='relu',padding='valid'))
  model.add(Conv2D(32,kernel_size=(5,5),activation='relu'))
  model.add(Conv2D(32,kernel_size=(5,5),activation='relu'))
  model.add(MaxPooling2D(pool_size=(3,3),strides=2))
  model.add(Dropout(0.5))
  model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(64,(1,1),activation='relu',padding='same'))
  model.add(Conv2D(64,(1,1),activation='relu',padding='same'))
  model.add(MaxPooling2D(pool_size=(3,3),strides=2))
  model.add(Dropout(0.5))
  model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
  model.add(Conv2D(32,(1,1),activation='relu'))
  model.add(Conv2D(5,(1,1)))
  model.add(GlobalAveragePooling2D())
  model.add(Flatten())
  model.add(Dense(5, activation='softmax'))
  model.build(input_shape)
  model.compile(loss=categorical_crossentropy,optimizer=Adam(0.001),metrics=['accuracy'])
  return model

In [ ]:
nin_model=nin_cnn_c(X,y)

In [ ]:
filepath = 'NIN_CNN_model.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,mode='auto', save_frequency=1)
print(X_train.shape, Y_train.shape)

(443, 124, 224, 3) (443, 5)


In [ ]:
history = model.fit(X_train, Y_train,batch_size = 2,epochs = 25,verbose = 1,validation_split = 0.2,validation_data = (X_test, Y_test),callbacks = [checkpoint],shuffle=True)

Epoch 1/25
176/177 [============================>.] - ETA: 0s - loss: 1.4315 - accuracy: 0.3864
Epoch 00001: val_loss improved from inf to 1.36544, saving model to NIN_CNN_model.hdf5
177/177 [==============================] - 5s 27ms/step - loss: 1.4302 - accuracy: 0.3870 - val_loss: 1.3654 - val_accuracy: 0.4944
Epoch 2/25
176/177 [============================>.] - ETA: 0s - loss: 1.4269 - accuracy: 0.3892
Epoch 00002: val_loss improved from 1.36544 to 1.36490, saving model to NIN_CNN_model.hdf5
177/177 [==============================] - 5s 28ms/step - loss: 1.4295 - accuracy: 0.3870 - val_loss: 1.3649 - val_accuracy: 0.4944
Epoch 3/25
175/177 [============================>.] - ETA: 0s - loss: 1.5752 - accuracy: 0.3829
Epoch 00003: val_loss improved from 1.36490 to 1.36186, saving model to NIN_CNN_model.hdf5
177/177 [==============================] - 5s 28ms/step - loss: 1.5712 - accuracy: 0.3842 - val_loss: 1.3619 - val_accuracy: 0.4944
Epoch 4/25
176/177 [===========================

Loading Models

In [ ]:
ConvPool_CNN_model=ConvPool_CNN_C(X,y)

In [ ]:
all_cnn_model=all_cnn_c(X,y)

In [ ]:
NIN_CNN_Model=nin_cnn_c(X,y)

Ensemble models

In [ ]:
Models=[ConvPool_CNN_model,all_cnn_model,NIN_CNN_Model]

In [ ]:
ConvPool_CNN_model_P=ConvPool_CNN_model.predict(X_test)
all_cnn_model_P= all_cnn_model.predict(X_test)
NIN_CNN_Model_P= NIN_CNN_Model.predict(X_test)
#averaged_preds = (ConvPool_CNN_model_P +all_cnn_model_P +NIN_CNN_Model_P )//3
